In [324]:
import pandas as pd
import numpy as np

# 1
Давайте проанализируем данные опроса 4361 женщин из Ботсваны:
botswana.tsv

О каждой из них мы знаем:

    сколько детей она родила (признак ceb)
    возраст (age)
    длительность получения образования (educ)
    религиозная принадлежность (religion)
    идеальное, по её мнению, количество детей в семье (idlnchld)
    была ли она когда-нибудь замужем (evermarr)
    возраст первого замужества (agefm)
    длительность получения образования мужем (heduc)
    знает ли она о методах контрацепции (knowmeth)
    использует ли она методы контрацепции (usemeth)
    живёт ли она в городе (urban)
    есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)

Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [325]:
data=pd.read_table('botswana.tsv')
data

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,0,16,7,protestant,2.0,1.0,0.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4357,2,28,7,protestant,4.0,1.0,1.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4358,4,24,5,protestant,4.0,1.0,1.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4359,1,26,0,spirit,5.0,1.0,0.0,1,22.0,7.0,0,0.0,1.0,0.0,0.0


In [326]:
data.religion.value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

# 2
Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски? 

In [327]:
data_without_nan=data.dropna()

In [328]:
data_without_nan.shape[0]

1834

# 3
В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному. 

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо". 

В подобных случаях, когда признак x1x_1x1​ на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

    создать новый бинарный признак x2={1,x1=’не применимо’,0,иначе;x_2 =
    1,x1='не применимо',0,иначе;
    x2​={1,0,​x1​=’не применимо’,иначе;​
    заменить "не применимо" в x1x_1x1​ на произвольную константу ccc, которая среди других значений x1x_1x1​ не встречается.

Теперь, когда мы построим регрессию на оба признака и получим модель вида y=β0+β1x1+β2x2,y=\beta_0 + \beta_1 x_1 + \beta_2 x_2,y=β0​+β1​x1​+β2​x2​, на тех объектах, где x1x_1x1​ было измерено, регрессионное уравнение примет вид y=β0+β1x,y=\beta_0 + \beta_1 x,y=β0​+β1​x, а там, где x1x_1x1​ было "не применимо", получится y=β0+β1c+β2.y=\beta_0 + \beta_1 c + \beta_2.y=β0​+β1​c+β2​. Выбор ccc влияет только на значение и интерпретацию β2\beta_2β2​, но не β1\beta_1β1​.

Давайте используем этот метод для обработки пропусков в agefm и heduc.

    Создайте признак nevermarr, равный единице там, где в agefm пропуски.
    Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице XXX будет мультиколлинеарность.
    Замените NaN в признаке agefm на cagefm=0c_{agefm}=0cagefm​=0.
    У объектов, где nevermarr = 1, замените NaN в признаке heduc на cheduc1=−1c_{heduc_1}=-1cheduc1​​=−1 (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).

Сколько осталось пропущенных значений в признаке heduc?

In [329]:
agefm=np.array(data.agefm)
print(agefm)
data['nevermarr']=agefm
data.nevermarr.fillna(1,inplace=True)
data

[nan 20. 22. ... nan 22. 21.]


,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0,20.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0,22.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,0,16,7,protestant,2.0,1.0,0.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0,1.0
4357,2,28,7,protestant,4.0,1.0,1.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0,1.0
4358,4,24,5,protestant,4.0,1.0,1.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0,1.0
4359,1,26,0,spirit,5.0,1.0,0.0,1,22.0,7.0,0,0.0,1.0,0.0,0.0,22.0


In [330]:
data.drop(['evermarr'],axis=1,inplace=True)
data

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,20.0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,22.0
3,0,24,12,other,2.0,1.0,0.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,0,16,7,protestant,2.0,1.0,0.0,NaN,NaN,0,0.0,1.0,0.0,0.0,1.0
4357,2,28,7,protestant,4.0,1.0,1.0,NaN,NaN,0,0.0,1.0,0.0,0.0,1.0
4358,4,24,5,protestant,4.0,1.0,1.0,NaN,NaN,0,0.0,1.0,0.0,0.0,1.0
4359,1,26,0,spirit,5.0,1.0,0.0,22.0,7.0,0,0.0,1.0,0.0,0.0,22.0


In [331]:
data['nevermarr'].loc[data.nevermarr>1]=0
data.nevermarr

/home/eketerina753/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0       1.0
1       0.0
2       0.0
3       1.0
4       0.0
       ... 
4356    1.0
4357    1.0
4358    1.0
4359    0.0
4360    0.0
Name: nevermarr, Length: 4361, dtype: float64

In [332]:
data.agefm.fillna(0,inplace=True)

In [333]:
data['heduc'].loc[data.nevermarr==1]=-1
data.heduc

0       -1.0
1       14.0
2        1.0
3       -1.0
4       12.0
        ... 
4356    -1.0
4357    -1.0
4358    -1.0
4359     7.0
4360     0.0
Name: heduc, Length: 4361, dtype: float64

In [334]:
data.heduc.isna().sum()

123

# Вопрос 4

Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения (cidlnchld=−1c_{idlnchld}=-1cidlnchld​=−1, cheduc2=−2c_{heduc_2}=-2cheduc2​​=−2 (значение -1 мы уже использовали), cusemeth=−1c_{usemeth}=-1cusemeth​=−1).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

### Create the indicator for idlnchld

In [335]:
data.idlnchld.isna().value_counts()

False    4241
True      120
Name: idlnchld, dtype: int64

In [336]:
idlnchld=data.idlnchld
data['idlnchld_noans']=idlnchld
data.idlnchld_noans.loc[data.idlnchld != 'NaN']=0
data.idlnchld_noans.loc[data.idlnchld.isna()]=1

/home/eketerina753/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [337]:
data.idlnchld_noans.value_counts()

0.0    4241
1.0     120
Name: idlnchld_noans, dtype: int64

### Create the indicator for heduc

In [338]:
data.heduc.isna().value_counts()

False    4238
True      123
Name: heduc, dtype: int64

In [339]:
heduc=data.heduc
data['heduc_noans']=heduc
data.heduc_noans.loc[data.heduc != 'NaN']=0
data.heduc_noans.loc[data.heduc.isna()]=1

In [340]:
data.heduc_noans.value_counts()

0.0    4238
1.0     123
Name: heduc_noans, dtype: int64

### Create the indicator for heduc usemeth

In [341]:
data.usemeth.isna().value_counts()

False    4290
True       71
Name: usemeth, dtype: int64

In [342]:
usemeth=data.usemeth
data['usemeth_noans']=usemeth
data.usemeth_noans.loc[data.usemeth != 'NaN']=0
data.usemeth_noans.loc[data.usemeth.isna()]=1

In [343]:
data.usemeth_noans.value_counts()

0.0    4290
1.0      71
Name: usemeth_noans, dtype: int64

### Заменяем пропуски на нехарактерные значения

In [344]:
data.idlnchld.isna().value_counts()

False    4241
True      120
Name: idlnchld, dtype: int64

In [345]:
data.idlnchld.loc[data.idlnchld.isna()]=-1

In [346]:
data.idlnchld.value_counts()

 4.0     1455
 6.0      718
 2.0      499
 5.0      490
 3.0      465
 10.0     166
 8.0      139
 7.0      125
-1.0      120
 1.0       95
 9.0       25
 0.0       21
 12.0      16
 20.0      11
 15.0       7
 11.0       3
 14.0       3
 13.0       1
 16.0       1
 17.0       1
Name: idlnchld, dtype: int64

In [347]:
data.heduc.isna().value_counts()

False    4238
True      123
Name: heduc, dtype: int64

In [348]:
data.heduc.loc[data.heduc.isna()]=-2

In [349]:
data.heduc.value_counts()

-1.0     2282
 0.0      692
 7.0      396
 10.0     210
-2.0      123
 6.0      102
 12.0      99
 4.0       73
 3.0       64
 5.0       62
 2.0       47
 9.0       35
 15.0      35
 14.0      29
 16.0      26
 1.0       25
 17.0      17
 8.0       17
 11.0       9
 13.0       8
 18.0       4
 19.0       4
 20.0       2
Name: heduc, dtype: int64

In [350]:
data.usemeth.isna().value_counts()

False    4290
True       71
Name: usemeth, dtype: int64

In [351]:
data.usemeth.loc[data.usemeth.isna()]=-1

In [352]:
data.usemeth.value_counts()

 1.0    2478
 0.0    1812
-1.0      71
Name: usemeth, dtype: int64

In [353]:
data.dropna(inplace=True)

In [354]:
data.shape

(4348, 18)

# 5
Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации R^2? Округлите до трёх знаков после десятичной точки.

In [355]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [356]:
#!pip install -U patsy

In [357]:
data.columns

Index(['ceb', 'age', 'educ', 'religion', 'idlnchld', 'knowmeth', 'usemeth',
       'agefm', 'heduc', 'urban', 'electric', 'radio', 'tv', 'bicycle',
       'nevermarr', 'idlnchld_noans', 'heduc_noans', 'usemeth_noans'],
      dtype='object')

In [358]:
m1 = smf.ols('ceb ~ age+educ+religion+idlnchld+knowmeth+usemeth+agefm+heduc+urban+electric+radio+tv+bicycle+nevermarr+idlnchld_noans+heduc_noans+usemeth_noans', 
             data=data)
fitted = m1.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Fri, 12 Mar 2021   Prob (F-statistic):               0.00
Time:                        06:45:08   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

# Вопрос 7

Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она

In [359]:
print( sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

1.1452927633439797e-225


# Вопрос 8

Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта. 

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.# 

In [360]:
m2 = smf.ols('ceb ~ age+educ+idlnchld+knowmeth+usemeth+agefm+heduc+urban+electric+bicycle+nevermarr+idlnchld_noans+heduc_noans+usemeth_noans', 
             data=data)
fitted2 = m2.fit()
print(fitted2.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Fri, 12 Mar 2021   Prob (F-statistic):               0.00
Time:                        06:45:09   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.

In [361]:
print( sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

1.1452927633439797e-225


=> модель гетероскедастична, необходимо сделать поправку Уайта

In [362]:
m3 = smf.ols('ceb ~ age+educ+idlnchld+knowmeth+usemeth+agefm+heduc+urban+electric+bicycle+nevermarr+idlnchld_noans+heduc_noans+usemeth_noans', 
             data=data)
fitted3 = m3.fit(cov_type='HC1')
print(fitted3.summary())


                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Fri, 12 Mar 2021   Prob (F-statistic):               0.00
Time:                        06:45:12   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [363]:
print("F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m3.fit()))

F=0.919236, p=0.467231, k1=5.000000


=> гипотезу о равенстве 0 коэффициентов удаленных признаков нельзя отвергнуть/

# 9
Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением cusemeth=−1c_{usemeth }=-1cusemeth​=−1, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили 5.5×10−85.5\times10^{-8}5.5×10−8, нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans.  

In [365]:
m4 = smf.ols('ceb ~ age+educ+idlnchld+knowmeth+agefm+heduc+urban+electric+bicycle+nevermarr+idlnchld_noans+heduc_noans', 
             data=data)
fitted4 = m4.fit(cov_type='HC1')
print(fitted4.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     396.4
Date:                Fri, 12 Mar 2021   Prob (F-statistic):               0.00
Time:                        06:51:45   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.262     -4.

In [369]:
print(m3.fit().compare_f_test(m4.fit())[1])

3.1552009480371243e-40


=> качество ухудшилось

print(fitted3.summary())

# Вопрос 10

Посмотрите на доверительные интервалы для коэффициентов итоговой модели (не забудьте использовать поправку Уайта, если есть гетероскедастичность ошибки) и выберите правильные выводы.

1.С увеличением возраста женщины на 1 год среднее количество детей возрастает на 0.17 (p<0.001, 95% доверительный интервал — [0.16, 0.18])


Да, т.к. coef при age =0.1702,  [0.162, 0.179]


2. У женщин, знакомых с методами контрацепции, при прочих равных в среднем на 0.6 ребёнка меньше (p=0.001, 95% доверительный интервал для разницы между средними — [-0.9, -0.2])


Нет, т.к. coef при usemeth = 0.6516  и доверительный интервал [0.550, 0.753]


3. У женщин, не знающих, какое количество детей идеально, в среднем на 0.66 ребёнка больше (p=0.002, 95% доверительный интервал — [0.2, 1.1])




Нет, см коэф при idlnchld_noans 

4. У женщин, не знающих, какое количество детей идеально, в среднем на βidlnchld_noans+cidlnchldβidlnchld≈0.58\beta_{idlnchld\_noans} + c_{idlnchld} \beta_{idlnchld} \approx 0.58 βidlnchld_noans​+cidlnchld​βidlnchld​≈0.58 детей больше

Да, подставляем в формулу

5. У женщин, никогда не выходивших замуж, при прочих равных в среднем на 2.3 ребёнка меньше (p<0.001, 95% доверительный интервал для разницы между средними — [-2.6, -1.9])






Да, см коэф при nevermarr


6. Итоговая модель объясняет 63% вариации отклика



Нет, коэффициент детерминации равен 0.644